In [1]:
# Import libraries
import pandas as pd
import os
from pathlib import Path
from tqdm import tqdm
import yaml
import matplotlib.pyplot as plt
from ultralytics.engine.results import Results
from ultralytics import YOLO
import numpy as np
from PIL import Image, ImageOps
import torch
from ultralytics.utils.patches import imread
import cv2

In [2]:
# INPUT_DIRS
INPUT_DATA_DIR = Path('dataset')
## Drop the Folder if it already exists
DATASETS_DIR = Path('dataset')
# Image & labels directory
TRAIN_IMAGES_DIR = DATASETS_DIR / 'images' / 'train'
TRAIN_LABELS_DIR = DATASETS_DIR / 'labels'/ 'train'
TEST_IMAGES_DIR = DATASETS_DIR / 'images' / 'test'
VAL_IMAGES_DIR = DATASETS_DIR / 'images' /'val'
VAL_LABELS_DIR = DATASETS_DIR / 'labels' /'val'

# Load train and test files
train = pd.read_csv(INPUT_DATA_DIR / 'Train_df.csv')
val = pd.read_csv(INPUT_DATA_DIR / 'Val_df.csv')
test = pd.read_csv(INPUT_DATA_DIR / 'Test.csv')
ss = pd.read_csv(INPUT_DATA_DIR / 'SampleSubmission.csv')

class_map = {cls: i for i, cls in enumerate(sorted(train['class'].unique().tolist()))}
# Strip any spacing from the class item and make sure that it is a str
train['class'] = train['class'].str.strip()

# Map {'healthy': 2, 'cssvd': 1, anthracnose: 0}
train['class_id'] = train['class'].map(class_map)

train_df = train
val_df = val

# Create a data.yaml file required by yolo
class_names = sorted(train['class'].unique().tolist())
num_classes = len(class_names)
data_yaml = {
	"path" : str(DATASETS_DIR.absolute()),
	'train': str(TRAIN_IMAGES_DIR.absolute()),
	'val': str(VAL_IMAGES_DIR.absolute()),
	'test': str(TEST_IMAGES_DIR.absolute()),
	'nc': num_classes,
	'names': class_names
}

val_image_names = [str(Path(name).stem) for name in val_df['Image_ID'].unique()]
train_image_names = [str(Path(name).stem) for name in train['ImagePath'].unique()]

In [3]:
from glob import glob

latest_run_dir = sorted(glob("zindi_challenge_cacao_stage2/train*"), key=lambda x: int(x.split('train')[-1]))[-1]

# Validate the model on the validation set
BEST_PATH = f"{latest_run_dir}/weights/best.pt"
# BEST_PATH = 'zindi_challenge_cacao_stage2/train10/weights/best.pt'
BEST_PATH

'zindi_challenge_cacao_stage2/train10/weights/best.pt'

In [4]:
from concurrent.futures import ThreadPoolExecutor

def load_image_(filepath):
	image = Image.open(filepath)
	# return image
	try:
		return ImageOps.exif_transpose(image)
	except Exception:
		pass
	return image


def load_image(filepath):
	return load_image_(filepath)
	# return load_image_(filepath)
	return imread(filepath, cv2.IMREAD_COLOR)

def load_images(filepaths):
	with ThreadPoolExecutor() as executor:
		images = list(executor.map(load_image_, filepaths))
	return images

In [5]:
# Validate the model on the validation set
BEST_CFG = f"{latest_run_dir}/args.yaml"
# BEST_CFG = 'zindi_challenge_cacao_stage2/train10/args.yaml'
BEST_CFG

'zindi_challenge_cacao_stage2/train10/args.yaml'

In [6]:
with open(BEST_CFG, 'r') as f:
	cfg: dict = yaml.safe_load(f)
	print(cfg)

{'task': 'detect', 'mode': 'train', 'model': 'zindi_challenge_cacao_stage2/train10/weights/last.pt', 'data': 'data.yaml', 'epochs': 136, 'time': 4.5, 'patience': 30, 'batch': 12, 'imgsz': 1024, 'save': True, 'save_period': -1, 'cache': False, 'device': '0,1', 'workers': 4, 'project': 'zindi_challenge_cacao_stage2', 'name': 'train10', 'exist_ok': False, 'pretrained': True, 'optimizer': 'auto', 'verbose': True, 'seed': 0, 'deterministic': False, 'single_cls': False, 'rect': False, 'cos_lr': False, 'close_mosaic': 10, 'resume': 'zindi_challenge_cacao_stage2/train10/weights/last.pt', 'amp': True, 'fraction': 1.0, 'profile': False, 'freeze': None, 'multi_scale': True, 'overlap_mask': True, 'mask_ratio': 4, 'dropout': 0.1, 'val': True, 'split': 'val', 'save_json': False, 'conf': None, 'iou': 0.6, 'max_det': 150, 'half': False, 'dnn': False, 'plots': True, 'source': None, 'vid_stride': 1, 'stream_buffer': False, 'visualize': False, 'augment': True, 'agnostic_nms': False, 'classes': None, 'ret

In [7]:
# Batch size for predictions
batch_size = 8

cfg["device"] = "cuda"
cfg["batch"] = batch_size
cfg["conf"] = 0.0
cfg["verbose"] = False
cfg["nms"] = True
cfg["iou"] = .6
cfg["agnostic_nms"] = False

cfg.pop("source", None)
# cfg.pop("batch_size")
cfg.pop("visualize", None)
cfg.pop("data", None)
cfg.pop("name", None)
# cfg.pop("half", None)

cfg["model"] = "predict"

keys = list(cfg.keys())
for col in keys:
    if (
        "show" in col  # Existing: removes show, show_labels, show_conf, show_boxes
        or "save" in col  # Existing: removes save, save_period, save_json, save_frames, save_txt, save_conf, save_crop, save_dir
        # or "freeze" in col  # Existing
        # Consider `col == 'nms'` instead of `"nms" in col` to avoid removing `agnostic_nms`
        # `agnostic_nms` is often useful for prediction.
        # or col == 'nms' # Removes the general nms flag if present
        # or "multi_scale" in col  # Existing
        or "plot" in col  # Existing
        # or "aug" in col  # Existing: removes augment, auto_augment. Also consider removing individual aug params if TTA is off.
        # or "drop" in col  # Existing
        # or "iou" in col  # Existing: removes training iou. Prediction uses its own iou parameter.
        or "lr" in col  # Existing: removes lr0, lrf, cos_lr, warmup_bias_lr
        or "mom" in col  # Existing: removes momentum, warmup_momentum
        or "wei" in col  # Existing: removes weight_decay
        # The 'half' parameter is crucial for mixed-precision inference.
        # If cfg['half'] is intended for prediction, this condition should not remove it.
        # or "half" in col # Existing: Problematic if 'half' is needed for prediction.
        # or "nbs" in col  # Existing
        # New conditions:
        or "epoch" in col  # Removes epochs, warmup_epochs
        or col == 'optimizer'
        or "worker" in col  # Removes workers
        # or col == 'val' or col == 'split' # Removes validation config from training
        or col == 'project' # Removes experiment project name
        # or col in ['box', 'cls', 'dfl', 'pose', 'kobj']  # Removes loss component weights
        # or col in ['format', 'keras', 'simplify', 'opset', 'int8', 'dynamic', 'workspace'] # Removes export-related params
        or col == 'patience'
        # or col == 'cache'
        # or col == 'seed'
        # or col == 'rect' # Rectangular training
        or col == 'resume'
        # or col == 'amp' # Training AMP flag (prediction uses 'half')
        or col == 'profile'
        or col == 'tracker'
        or col == 'task'
        or col == 'mode' # e.g., mode: train
        or col == 'pretrained'
        or col == 'deterministic'
        or col == 'exist_ok'
        # or col == 'single_cls'
        or col == 'time' # training time limit
        or col == 'cfg' # path to model cfg yaml (e.g., yolov8n.yaml)
        # If 'augment' key is removed (disabling Test Time Augmentation),
        # you might also want to remove individual augmentation parameters:
        # or col in ['degrees', 'translate', 'scale', 'shear', 'perspective', 'flipud', 'fliplr', 'bgr', 'mosaic', 'mixup', 'cutmix', 'copy_paste', 'erasing']
        # or col.startswith('hsv_') # hsv_h, hsv_s, hsv_v
    ):
        cfg.pop(col)

print(cfg)

{'model': 'predict', 'batch': 8, 'imgsz': 1024, 'cache': False, 'device': 'cuda', 'verbose': False, 'seed': 0, 'single_cls': False, 'rect': False, 'close_mosaic': 10, 'amp': True, 'fraction': 1.0, 'freeze': None, 'multi_scale': True, 'overlap_mask': True, 'mask_ratio': 4, 'dropout': 0.1, 'val': True, 'split': 'val', 'conf': 0.0, 'iou': 0.6, 'max_det': 150, 'half': False, 'dnn': False, 'vid_stride': 1, 'stream_buffer': False, 'augment': True, 'agnostic_nms': False, 'classes': None, 'retina_masks': False, 'embed': None, 'line_width': None, 'format': 'torchscript', 'keras': False, 'optimize': False, 'int8': False, 'dynamic': False, 'simplify': True, 'opset': None, 'workspace': None, 'nms': True, 'box': 7.5, 'cls': 1.0, 'dfl': 1.5, 'pose': 12.0, 'kobj': 1.0, 'nbs': 64, 'hsv_h': 0.015, 'hsv_s': 0.7, 'hsv_v': 0.4, 'degrees': 0.0, 'translate': 0.1, 'scale': 0.5, 'shear': 0.0, 'perspective': 0.0, 'flipud': 0.3, 'bgr': 0.0, 'mosaic': 1.0, 'mixup': 0.1, 'cutmix': 0.0, 'copy_paste': 0.1, 'copy_pa

In [ ]:
# Load the trained YOLO model
model = YOLO(BEST_PATH)

# Path to the test images directory
test_dir_path = TEST_IMAGES_DIR

# Get a list of all image files in the test directory
image_files = os.listdir(test_dir_path)

# Initialize an empty list to store the results for all images
all_data = []

# Initialize an empty list to store the results for all images
all_data = []

# Batch size for predictions
batch_size = 16

# Process images in batches
for i in tqdm(range(0, len(image_files), batch_size)):
    batch_files = image_files[i : i + batch_size]
    batch_images = load_images(
        [os.path.join(test_dir_path, img_file) for img_file in batch_files]
    )  # [load_image(os.path.join(test_dir_path, img_file)) for img_file in batch_files]

    # Make predictions on the batch of images
    results = model.predict(
        batch_images,
        **cfg,
    )

    # Iterate through each result in the batch
    for img_file, result in zip(batch_files, results):
        boxes = (
            result.boxes.xyxy.tolist() if result.boxes else []
        )  # Bounding boxes in xyxy format
        classes = result.boxes.cls.tolist() if result.boxes else []  # Class indices
        confidences = (
            result.boxes.conf.tolist() if result.boxes else []
        )  # Confidence scores
        names = result.names  # Class names dictionary

        if boxes:  # If detections are found
            for box, cls, conf in zip(boxes, classes, confidences):
                x1, y1, x2, y2 = box
                detected_class = names[
                    int(cls)
                ]  # Get the class name from the names dictionary

                # Add the result to the all_data list
                all_data.append(
                    {
                        "Image_ID": str(img_file),
                        "class": detected_class,
                        "confidence": conf,
                        "ymin": y1,
                        "xmin": x1,
                        "ymax": y2,
                        "xmax": x2,
                    }
                )
        else:  # If no objects are detected
            all_data.append(
                {
                    "Image_ID": str(img_file),
                    "class": "None",
                    "confidence": None,
                    "ymin": None,
                    "xmin": None,
                    "ymax": None,
                    "xmax": None,
                }
            )

  0%|          | 0/102 [00:00<?, ?it/s]

 13%|█▎        | 13/102 [00:43<04:32,  3.07s/it]

In [ ]:
# Convert the list to a DataFrame for all images
sub = pd.DataFrame(all_data)

In [ ]:
sub.head()

,Image_ID,class,confidence,ymin,xmin,ymax,xmax
0,ID_cWEAQI.jpeg,healthy,0.627020,11.463284,2.905130,3970.572754,1657.665161
1,ID_cWEAQI.jpeg,healthy,0.070782,183.179245,0.000000,1397.601196,475.406128
2,ID_cWEAQI.jpeg,healthy,0.037787,343.278229,1375.410278,1119.676636,1796.921387
3,ID_cWEAQI.jpeg,anthracnose,0.036683,2823.884277,525.201538,3995.499268,1549.878784
4,ID_cWEAQI.jpeg,anthracnose,0.009041,2251.175781,0.000000,2764.001709,389.630280


In [ ]:
sub.describe()

,confidence,ymin,xmin,ymax,xmax
count,162600.000000,162600.000000,162600.000000,162600.000000,162600.000000
mean,0.015198,744.563120,684.176535,1358.567956,1264.352718
std,0.092043,954.264246,774.782582,1165.236030,949.777946
min,0.000008,0.000000,0.000000,0.000000,0.000000
25%,0.000090,1.853634,50.530201,383.166565,524.856201
50%,0.000279,338.495468,425.593369,1070.792053,960.000000
75%,0.001159,1145.714844,1012.358185,2047.754120,1813.651459
max,0.931821,4064.660400,4051.008789,4128.000000,4128.000000


In [ ]:
sub['class'].value_counts()

class
cssvd          64732
healthy        54884
anthracnose    42984
Name: count, dtype: int64

In [ ]:
sub.isna().sum()

Image_ID      0
class         0
confidence    0
ymin          0
xmin          0
ymax          0
xmax          0
dtype: int64

class
healthy        1153
cssvd           801
anthracnose     694
None             57
Name: count, dtype: int6

In [ ]:
sub.to_csv("dataset/predictions/11-predictions.csv", index=False)

In [ ]:
sub["confidence"].describe()

count    162600.000000
mean          0.015198
std           0.092043
min           0.000008
25%           0.000090
50%           0.000279
75%           0.001159
max           0.931821
Name: confidence, dtype: float64

In [ ]:
import pandas as pd

sub = pd.read_csv('dataset/predictions/11-predictions.csv')

sub.sample(6)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax
156801,ID_YX6bNA.jpg,cssvd,0.000405,993.928894,0.000000,1280.000000,0.000000
38893,ID_hBZYGx.jpg,anthracnose,0.000124,1316.450439,641.409546,2012.599365,1201.425415
141377,ID_d6gpj1.jpg,cssvd,0.000046,0.000000,443.865082,681.580566,774.209351
155251,ID_aY2yXb.jpg,anthracnose,0.000384,5.117552,1159.703125,389.365021,1486.335205
12633,ID_JgQ193.jpg,anthracnose,0.000381,435.697357,740.825317,857.347534,960.000000
36032,ID_Nbmn82.jpg,cssvd,0.000164,1975.368286,713.659424,2047.494263,1160.406250


In [ ]:
sub["Image_ID"].value_counts().describe()

count    1626.0
mean      100.0
std         0.0
min       100.0
25%       100.0
50%       100.0
75%       100.0
max       100.0
Name: count, dtype: float64

In [ ]:
sub["Image_ID"].nunique()

1626

In [ ]:
sub.isna().sum()

Image_ID      0
class         0
confidence    0
ymin          0
xmin          0
ymax          0
xmax          0
dtype: int64